In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression


In [14]:
dataset=pd.read_csv('laptop_price - dataset.csv')
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)
indep_X=dataset.drop('Price (Euro)', axis='columns')
dep_Y=dataset['Price (Euro)']

In [19]:
def selectkbest(indep_X, dep_Y, n):
    from sklearn.feature_selection import SelectKBest
    test=SelectKBest(score_func=f_regression, k=n)
    fit1=test.fit(indep_X, dep_Y)
    selectk_features=fit1.transform(indep_X)
    return selectk_features

def split_scaler(indep_X, dep_Y):
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import StandardScaler
    x_train, x_test, y_train, y_test=train_test_split(indep_X, dep_Y, test_size=0.25, random_state=0)
    sc=StandardScaler()
    x_train=sc.fit_transform(x_train)
    x_test=sc.transform(x_test)
    return x_train, x_test, y_train, y_test

def r2_score_prediction(regressor, x_test, y_test):
    from sklearn.metrics import r2_score
    y_pred=regressor.predict(x_test)    
    r2_score=r2_score(y_test, y_pred)
    return r2_score, regressor, y_pred

def linear(x_train, y_train, x_test, y_test):
    from sklearn.linear_model import LinearRegression
    regressor=LinearRegression()
    regressor.fit(x_train, y_train)
    r2_score, regressor, y_pred=r2_score_prediction(regressor, x_test, y_test)
    return r2_score, regressor, y_pred

def svr(x_train, y_train, x_test, y_test):
    from sklearn.svm import SVR
    regressor=SVR(kernel="sigmoid", C=10)
    regressor.fit(x_train, y_train)
    r2_score, regressor, y_pred=r2_score_prediction(regressor, x_test, y_test)
    return r2_score, regressor, y_pred

def decisiontree(x_train, y_train, x_test, y_test):
    from sklearn.tree import DecisionTreeRegressor
    regressor=DecisionTreeRegressor(criterion="absolute_error", splitter="random")
    regressor=regressor.fit(x_train, y_train)
    r2_score, regressor, y_pred=r2_score_prediction(regressor, x_test, y_test)
    return r2_score, regressor, y_pred

def randomforest(x_train, y_train, x_test, y_test):
    from sklearn.ensemble import RandomForestRegressor
    regressor=RandomForestRegressor(n_estimators=50, random_state=0)
    regressor.fit(x_train, y_train)
    r2_score, regressor, y_pred=r2_score_prediction(regressor, x_test, y_test)
    return r2_score, regressor, y_pred

def selectk_Regression(acclin, accsvm, accdec, accrf):
    dataframe=pd.DataFrame(index=['f_regression'], columns=['Linear', 'SVM', 'Decision', 'Random'])
    for number, idex in enumerate(dataframe.index):
        dataframe['Linear'][idex]=acclin[number]
        dataframe['SVM'][idex]=accsvm[number]
        dataframe['Decision'][idex]=accdec[number]
        dataframe['Random'][idex]=accrf[number]
    return dataframe

In [36]:
kbest=selectkbest(indep_X, dep_Y, 10)
acclin=[]
accsvm=[]
accdec=[]
accrf=[]

x_train, x_test, y_train, y_test=split_scaler(kbest, dep_Y)
r2_score, regressor, y_pred=linear(x_train, y_train, x_test, y_test)
acclin.append(r2_score)
r2_score, regressor, y_pred=svr(x_train, y_train, x_test, y_test)
accsvm.append(r2_score)
r2_score, regressor, y_pred=decisiontree(x_train, y_train, x_test, y_test)
accdec.append(r2_score)
r2_score, regressor, y_pred=randomforest(x_train, y_train, x_test, y_test)
accrf.append(r2_score)
result=selectk_Regression(acclin, accsvm, accdec, accrf)

C:\Users\jspri\AppData\Local\Temp\ipykernel_23220\2545720157.py:54: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  dataframe['Linear'][idex]=acclin[number]
C:\Users\jspri\AppData\Local\Temp\ipykernel_23220\2545720157.py:55: FutureWarning: Cha

In [34]:
import pickle
filename="finalized_model_random_forest.sav"

In [37]:
r2_score, regressor, y_pred = randomforest(x_train, y_train, x_test, y_test)
acclin.append(r2_score)
with open('finalized_model_random_forest.sav', 'wb') as file:
    pickle.dump(regressor, file)

In [41]:
loaded_model=pickle.load(open("finalized_model_random_forest.sav", 'rb'))
result=loaded_model.predict([[13.3, 2.3, 8, 1.37, 1, 1, 0, 1, 1, 1]])

In [43]:
result

array([1529.62855952])